In [12]:
# Necessary Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision


In [2]:
# Define Encoder and Decoder 

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.t_conv1 = nn.ConvTranspose2d(4, 8, kernel_size=2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(8, 16, kernel_size=2, stride=2)
        self.t_conv3 = nn.ConvTranspose2d(16, 3, kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.t_conv1(x)
        x = nn.functional.relu(x)
        x = self.t_conv2(x)
        x = nn.functional.relu(x)
        x = self.t_conv3(x)
        x = nn.functional.sigmoid(x)
        return x


In [3]:
# Define AutoEncoder 

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [4]:
# Define Loss function and optimizer
model = AutoEncoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
# Load images and Preprocess 

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


dataset = torchvision.datasets.Caltech101(root='path/to/Caltech101', transform=transform,download=True)

# Load dataset and apply transformation 
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)


# Set train and test size
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Train test split 80-20 %
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load train and test dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


0it [00:00, ?it/s]

Extracting path/to/Caltech101\caltech101\101_ObjectCategories.tar.gz to path/to/Caltech101\caltech101


In [ ]:
# Train AE 
num_epochs = 10

for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, img)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}:")
